In [129]:
import numpy as np
import pandas as pd
import datatable
import itertools as it
from owlready2 import *
import math
import joblib

import pickle
from bayes_opt import BayesianOptimization
#para depurar los que letras que no están en el codigo ascii
import unicodedata
import functools
import spacy
import stanfordnlp
#from spacy_stanfordnlp import StanfordNLPLanguage
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import metrics
#from wiki_dump_reader import Cleaner, iterate
from sklearn.decomposition import TruncatedSVD

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize,scale



#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import SVG

import random
from sklearn.externals import joblib


#Operaciones con gráficos
import scattertext as st
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

import networkx as nx

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool, LabelSet, ColumnDataSource, Range1d
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE

#import gensim
#import gensim.corpora as corpora
#from gensim.models import CoherenceModel
#from gensim.models import HdpModel

warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from multiprocessing import  Pool
import math
import scipy.sparse as sp


from sklearn.model_selection import *
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix



In [3]:
import json

with open('../config/config.json', 'r') as f:
    config = json.load(f)


In [4]:
corpus = pd.read_csv('../data/interim/amazon/corpus_normalizado_t_original.csv')
indices = pd.read_csv('../data/interim/amazon/corpus1_50k.csv')
indices = indices['Unnamed: 0']
corpus = corpus.loc[indices,:]
corpus = corpus.iloc[:,[3,5]]
corpus.head()

,normalizado,target_original
9,book recommend friend finally read year late w...,1
10,need question know child love impair identify ...,1
11,keep interest way definitely read jacob s moth...,1
51,c w gortner favorite author time wow queen rel...,1
61,love book like jk alex delaware series problem...,1


In [5]:
corpus.to_csv('../data/interim/amazon/corpus_normalizado.csv')

interimpath = '../data/interim/amazon/corpus_normalizado.csv'

In [6]:
interimpath

'../data/interim/amazon/corpus_normalizado.csv'

In [9]:
corpus = pd.read_csv(interimpath)
corpus.head()

,Unnamed: 0,normalizado,target_original
0,9,book recommend friend finally read year late w...,1
1,10,need question know child love impair identify ...,1
2,11,keep interest way definitely read jacob s moth...,1
3,51,c w gortner favorite author time wow queen rel...,1
4,61,love book like jk alex delaware series problem...,1


In [12]:

corpus = corpus.loc[:,['normalizado','target_original']]
corpus.columns=['cleaned_text','Target']
corpus.Target = corpus.Target.astype(int)
corpus.head()

,cleaned_text,Target
0,book recommend friend finally read year late w...,1
1,need question know child love impair identify ...,1
2,keep interest way definitely read jacob s moth...,1
3,c w gortner favorite author time wow queen rel...,1
4,love book like jk alex delaware series problem...,1


In [13]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


In [14]:
stop_words.append('be')
stop_words.append('don')
from sklearn.feature_extraction import text
my_stop_words = text.ENGLISH_STOP_WORDS.union(["be","don"])


In [8]:
random.seed(100)

Para hacer LDA, no vamos a utilizar todo el corpus. Seleccionaremos aleatoriamente 100.000 textos de clase positiva y sin clasificar para posteriormente hacer LDA.

In [15]:
tfidf = TfidfVectorizer(min_df=50,max_df=0.95, stop_words=['english','spanish'], ngram_range=(1,1))
#por ahora no pongo ngramas
joblib.dump(tfidf,'../objects/'+'amazon'+'/tfidf_object')

['../objects/amazon/tfidf_object']

In [16]:
dtm = tfidf.fit_transform(corpus.iloc[:,0])

In [17]:
dtm

<50000x3468 sparse matrix of type '<class 'numpy.float64'>'
	with 1209577 stored elements in Compressed Sparse Row format>

### Anteriormente, hemos realizado un grid search. Vamos a utilizar los parámetros que mejor funcionaron en su momento

from sklearn.model_selection import GridSearchCV
search_params = { 'topic_word_prior':[0.5,0.7,0,0.95,0.99], # hemos partido de 0.5, 0.7, 0,9, 0.95 y 0.99
                 'doc_topic_prior':[0.01,0.015,0.05,0.3,0.5]}

LDA = LatentDirichletAllocation(n_components=2,random_state=42,n_jobs=10,batch_size=50000,  evaluate_every = 100,  mean_change_tol=0.05)

model = GridSearchCV(LDA, param_grid=search_params)
from sklearn.utils import parallel_backend
i = time.time()
with parallel_backend('threading'):
    # Do the Grid Search
    model.fit(dtm)

    joblib.dump(model, 'lda_model4.pkl')

f = time.time()
elapsed = f - i
print('transcurridos {0} s'.format(elapsed))


## Ya sabemos que nuestro mejor modelo tiene unos priors determinados, 
joblib.dump(modelo,'../objects/'+config.get('corpus')+'/modelo_gridsearch.final')


In [18]:
LDA = LatentDirichletAllocation(n_components=2,random_state=42,topic_word_prior = 0.95,doc_topic_prior = 0.015,n_jobs=10,max_iter=250,mean_change_tol=0.05)
joblib.dump(LDA,'../objects/amazon/lda.final')


['../objects/amazon/lda.final']

In [19]:
#tfidf = joblib.load('tfidf')
tfidf

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.95, max_features=None, min_df=50,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=['english', 'spanish'], strip_accents=None,
        sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, use_idf=True, vocabulary=None)

In [20]:
texts = corpus.iloc[:,0]
texts = list(texts.values)

In [21]:
feature_names = tfidf.get_feature_names()

In [22]:
feature_names

['00',
 '000',
 '10',
 '100',
 '1000',
 '1080p',
 '11',
 '12',
 '120',
 '13',
 '14',
 '15',
 '150',
 '16',
 '17',
 '18',
 '19',
 '1st',
 '20',
 '200',
 '2000',
 '2008',
 '2009',
 '2010',
 '2011',
 '2012',
 '2013',
 '2014',
 '2015',
 '20th',
 '21',
 '22',
 '23',
 '24',
 '25',
 '250',
 '26',
 '27',
 '28',
 '2nd',
 '30',
 '300',
 '32',
 '35',
 '39',
 '3rd',
 '40',
 '400',
 '45',
 '4th',
 '50',
 '500',
 '55',
 '5th',
 '60',
 '64',
 '65',
 '70',
 '75',
 '80',
 '800',
 '90',
 '95',
 '99',
 'aa',
 'abandon',
 'ability',
 'able',
 'absolute',
 'absolutely',
 'absorb',
 'abuse',
 'abusive',
 'ac',
 'accept',
 'acceptable',
 'access',
 'accessible',
 'accessory',
 'accident',
 'accidentally',
 'accompany',
 'accomplish',
 'accord',
 'account',
 'accuracy',
 'accurate',
 'accurately',
 'accuse',
 'achieve',
 'acknowledge',
 'acquire',
 'act',
 'action',
 'active',
 'activity',
 'actor',
 'actual',
 'actually',
 'ad',
 'adam',
 'adapt',
 'adapter',
 'add',
 'addict',
 'addiction',
 'addition',
 'a

In [23]:
i = time.time()
from sklearn.utils import parallel_backend
i = time.time()
with parallel_backend('threading'):
    modelo = LDA.fit(dtm)
print (' terminado en {0} min'.format((time.time() - i) / 60))

 terminado en 40.55226482550303 min


In [28]:
joblib.dump(modelo,'../objects/amazon/final_lda_fit.final.pck')
joblib.dump(feature_names,'../objects/amazon/feature_names.final')
joblib.dump(tfidf,'../objects/amazon/tfidf.final')

['../objects/amazon/tfidf.final']

In [20]:
#modelo = joblib.load('final_lda.pck')
#modelo = joblib.load('final_lda.final.pck')


In [1]:
 from sklearn.externals import joblib
modelo = joblib.load('../objects/amazon/final_lda_fit.final.pck')

In [2]:
tfidf = joblib.load('../objects/amazon/tfidf_object')

In [3]:
feature_names = joblib.load('../objects/amazon/feature_names.final')

In [26]:
for index,topic in enumerate(modelo.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([feature_names[i] for i in topic.argsort()[-15:]])
    joblib.dump([feature_names[i] for i in topic.argsort()],'../objects/amazon/topics'+str(index))
    print('\n')
    

THE TOP 15 WORDS FOR TOPIC #0
['fit', 'dog', 'need', 'nice', 'easy', 'like', 'love', 'quality', 'price', 'buy', 'use', 'good', 'product', 'great', 'work']


THE TOP 15 WORDS FOR TOPIC #1
['time', 'end', 'interest', 'author', 'enjoy', 'write', 'like', 'character', 'good', 'great', 'series', 'love', 'story', 'read', 'book']




In [ ]:
###original

In [27]:
for index,topic in enumerate(modelo.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    joblib.dump([tfidf.get_feature_names()[i] for i in topic.argsort()],'../objects/amazon/topics'+str(index))
    print('\n')
    

THE TOP 15 WORDS FOR TOPIC #0
['fit', 'dog', 'need', 'nice', 'easy', 'like', 'love', 'quality', 'price', 'buy', 'use', 'good', 'product', 'great', 'work']


THE TOP 15 WORDS FOR TOPIC #1
['time', 'end', 'interest', 'author', 'enjoy', 'write', 'like', 'character', 'good', 'great', 'series', 'love', 'story', 'read', 'book']




In [ ]:
##aqui

In [22]:
#save dtm as dataframe

In [127]:
pds = pd.DataFrame(dtm.todense(), columns = feature_names)

In [173]:
joblib.dump(dtm,'../objects/amazon/dtm.final')

['../objects/amazon/dtm.final']

In [30]:
joblib.dump(pds,'../objects/amazon/dtm_pds')

['../objects/amazon/dtm_pds']

In [31]:
ldares = modelo.transform(dtm)

In [32]:
joblib.dump(ldares,'../objects/amazon/ldares_transform.final')

['../objects/amazon/ldares_transform.final']

In [33]:
ldares = joblib.load('../objects/amazon/ldares_transform.final')

In [34]:
ldares

array([[0.00303975, 0.99696025],
       [0.00384368, 0.99615632],
       [0.00463416, 0.99536584],
       ...,
       [0.99515363, 0.00484637],
       [0.99648833, 0.00351167],
       [0.99373686, 0.00626314]])

In [160]:
def xgb_cv(max_depth, ntrees,min_rows,eta,learn_rate,sample_rate,colsample_bytree,reg_lambda,reg_alpha,train):
    """SVC cross validation.
    This function will instantiate a SVC classifier with parameters C and
    gamma. Combined with data and targets this will in turn be used to perform
    cross validation. The result of cross validation is returned.
    Our goal is to find combinations of C and gamma that maximizes the roc_auc
    metric.
    """
    params = {'max_depth': int(max_depth), 'ntrees': int(ntrees),'min_rows':int(min_rows),'eta':float(eta),
              'learn_rate':float(learn_rate),'sample_rate':float(sample_rate),
              'colsample_bytree':float(colsample_bytree),'reg_lambda':float(reg_lambda),'reg_alpha':float(reg_alpha)}
      
    estimator = H2OXGBoostEstimator(**params)
    train['target'] = train['target'].asfactor()
    test['target'] = test['target'].asfactor()
    
    estimator.train(x=['A','B'],y='target',training_frame=train,validation_frame=test)
    return estimator.auc()

def optimize_xgb(train):
    """Apply Bayesian Optimization to XGB parameters."""
    def xgb_crossval(max_depth, ntrees,min_rows,eta,learn_rate,sample_rate,colsample_bytree,reg_lambda,reg_alpha,train=train):
        """Wrapper of SVC cross validation.
        Notice how we transform between regular and log scale. While this
        is not technically necessary, it greatly improves the performance
        of the optimizer.
        """
        return xgb_cv(max_depth, ntrees,min_rows,eta,learn_rate,sample_rate,colsample_bytree,reg_lambda,reg_alpha,train)

    optimizer = BayesianOptimization(
        f=xgb_crossval,
        
        pbounds={"max_depth": (4, 40), "ntrees": (50,400),'min_rows':(2,50),'eta':(0.01,1),
                 'learn_rate':(0.01,1),'sample_rate':(0.1,0.5),'colsample_bytree':(0.2,0.8),
                 'reg_lambda':(0,1),'reg_alpha':(0,1)},
        random_state=1234,
        verbose=2
    )
    modelos = optimizer.maximize(n_iter=10)
    return modelos

In [36]:
data = pd.DataFrame(ldares,columns=['A','B'])
data['Target'] = corpus.iloc[:,1]

In [49]:
data.to_csv('../data/interim/amazon/datatopic.csv')

25000

In [51]:
import  h2o

In [52]:
from h2o.estimators import H2OXGBoostEstimator, H2ODeepLearningEstimator,H2ORandomForestEstimator,H2OSupportVectorMachineEstimator

In [53]:

h2o.init(max_mem_size='14G')

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.6" 2020-01-14; OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1); OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu118.04.1, mixed mode, sharing)
  Starting server from /home/rdelaguila/anaconda3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpvix50en2
  JVM stdout: /tmp/tmpvix50en2/h2o_rdelaguila_started_from_python.out
  JVM stderr: /tmp/tmpvix50en2/h2o_rdelaguila_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Berlin
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,5 months and 10 days !!!
H2O cluster name:,H2O_from_python_rdelaguila_2bi8mt
H2O cluster total nodes:,1
H2O cluster free memory:,14 Gb
H2O cluster total cores:,12
H2O cluster allowed cores:,12
H2O cluster status:,"accepting new members, healthy"


In [54]:
frame = h2o.upload_file('../data/interim/amazon/datatopic.csv',sep=',',header=1)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [55]:
frame.head()

C1,A,B,Target
0,0.00303975,0.99696,1
1,0.00384368,0.996156,1
2,0.00463416,0.995366,1
3,0.00157954,0.99842,1
4,0.0041268,0.995873,1
5,0.00440491,0.995595,1
6,0.0048532,0.995147,1
7,0.00433073,0.995669,1
8,0.00413952,0.99586,1
9,0.00371882,0.996281,1


In [56]:
frame = frame.drop('C1')

In [58]:
frame['target']=frame['Target'].asfactor()
train,test=frame.split_frame(ratios=[.75])

In [162]:
modelos = optimize_xgb(train)

|   iter    |  target   | colsam... |    eta    | learn_... | max_depth | min_rows  |  ntrees   | reg_alpha | reg_la... | sample... |
-------------------------------------------------------------------------------------------------------------------------------------
xgboost Model Build progress: |███████████████████████████████████████████| 100%
|  1        |  0.9421   |  0.3149   |  0.6259   |  0.4434   |  32.27    |  39.44    |  145.4    |  0.2765   |  0.8019   |  0.4833   |
xgboost Model Build progress: |███████████████████████████████████████████| 100%
|  2        |  0.9226   |  0.7256   |  0.3642   |  0.506    |  28.6     |  36.21    |  179.6    |  0.5612   |  0.5031   |  0.1055   |
xgboost Model Build progress: |███████████████████████████████████████████| 100%
|  3        |  0.9663   |  0.6637   |  0.8838   |  0.3712   |  26.15    |  5.618    |  179.1    |  0.9331   |  0.6514   |  0.2589   |
xgboost Model Build progress: |███████████████████████████████████████████| 100%
|  4  

In [165]:
params = {'max_depth': int(39), 'ntrees': int(393),'min_rows':int(5),'eta':float(0.6331),
              'learn_rate':float(0.418),'sample_rate':float(0.2508),
              'colsample_bytree':float(0.633),'reg_lambda':float(0.8628),'reg_alpha':float(0.06257)}
      
estimator = H2OXGBoostEstimator(**params)
train['target'] = train['target'].asfactor()
test['target'] = test['target'].asfactor()
    
estimator.train(x=['A','B'],y='target',training_frame=train,validation_frame=test)

xgboost Model Build progress: |███████████████████████████████████████████| 100%


In [168]:
performance = estimator.model_performance(test)
performance

In [193]:
performance


ModelMetricsBinomial: xgboost
** Reported on test data. **

MSE: 0.11759757685304478
RMSE: 0.3429250309514381
LogLoss: 0.44860056941989046
Mean Per-Class Error: 0.13632630714481242
AUC: 0.8978100929556074
AUCPR: 0.8510384917558662
Gini: 0.7956201859112149

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2780561298131943: 


,,0,1,Error,Rate
0,0,4929.0,1253.0,0.2027,(1253.0/6182.0)
1,1,453.0,5904.0,0.0713,(453.0/6357.0)
2,Total,5382.0,7157.0,0.1361,(1706.0/12539.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.278056,0.873761,283.0
1,max f2,0.083471,0.916063,341.0
2,max f0point5,0.538910,0.852487,211.0
3,max accuracy,0.311470,0.864503,275.0
4,max precision,0.999501,0.906977,0.0
5,max recall,0.000922,1.000000,398.0
6,max specificity,0.999501,0.999353,0.0
7,max absolute_mcc,0.300916,0.733600,278.0
8,max min_per_class_accuracy,0.568096,0.848107,202.0
9,max mean_per_class_accuracy,0.311470,0.863674,275.0



Gains/Lift Table: Avg response rate: 50.70 %, avg score: 50.64 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.010288,0.998292,1.788986,1.788986,0.906977,0.998909,0.906977,0.998909,0.018405,0.018405,78.898559,78.898559
1,,2,0.020337,0.997351,1.612417,1.701740,0.817460,0.997867,0.862745,0.998394,0.016203,0.034608,61.241701,70.173994
2,,3,0.030066,0.996569,1.778458,1.726566,0.901639,0.996884,0.875332,0.997906,0.017304,0.051911,77.845772,72.656638
3,,4,0.040195,0.995519,1.677377,1.714171,0.850394,0.995992,0.869048,0.997423,0.016989,0.068900,67.737716,71.417148
4,,5,0.050004,0.994547,1.828144,1.736530,0.926829,0.995005,0.880383,0.996949,0.017933,0.086833,82.814412,73.652975
5,,6,0.100008,0.989499,1.730238,1.733384,0.877193,0.992002,0.878788,0.994476,0.086519,0.173352,73.023797,73.338386
6,,7,0.150251,0.983800,1.700082,1.722248,0.861905,0.986855,0.873142,0.991927,0.085418,0.258770,70.008240,72.224802
7,,8,0.200973,0.975951,1.730564,1.724347,0.877358,0.979845,0.874206,0.988878,0.087777,0.346547,73.056443,72.434693
8,,9,0.300024,0.942369,1.675489,1.708217,0.849436,0.962482,0.866029,0.980163,0.165959,0.512506,67.548890,70.821676
9,,10,0.400112,0.857930,1.695854,1.705124,0.859761,0.907741,0.864461,0.962047,0.169734,0.682240,69.585380,70.512418


In [168]:
estimator.save_mojo('..objects/amazon/xgb_model_topic')

'/mnt/c/Users/Raúl/Documents/Doctorado/curso NLP/FINAL-NLP-COURSE/FINAL_NLP_COURSE/Scripts Phd/oboe/notebooks/..objects/amazon/xgb_model_topic/XGBoost_model_python_1594832434245_5585.zip'

In [69]:
X_train, X_test, y_train, y_test = train_test_split(data.loc[:,['A','B']], data['Target'], test_size = 0.30)
svm = SVC(C=10**2 ,gamma=10**-4,probability=True)


svmlda = svm.fit(X_train,y_train)

In [70]:
svmlda

SVC(C=100, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.0001, kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [71]:
y_pred = svmlda.predict(X_test)


In [72]:
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[6317 1129]
 [ 525 7029]]
              precision    recall  f1-score   support

           0       0.92      0.85      0.88      7446
           1       0.86      0.93      0.89      7554

   micro avg       0.89      0.89      0.89     15000
   macro avg       0.89      0.89      0.89     15000
weighted avg       0.89      0.89      0.89     15000



In [9]:
#Nota: this is a previous evaluation metric that helps us to infer how the topics are distributed again the first target, but must not be considered along the process

In [78]:
joblib.dump(svmlda,'../objects/amazon/svmldatmp.final')

['../objects/amazon/svmldatmp.final']

In [ ]:
##aqui

In [170]:
joblib.dump(corpus.iloc[:,1],'../objects/amazon/y_a_predecir')

['../objects/amazon/y_a_predecir']

<50000x3468 sparse matrix of type '<class 'numpy.float64'>'
	with 1209577 stored elements in Compressed Sparse Row format>

# Analisis de anomalias

## Añadimos probabilidad de pertenencia


In [ ]:
import joblib
LDA = joblib.load('../objects/amazon/LDA.final')
dtm = joblib.load('../objects/amazon/dtm.final')

In [174]:
topic_results = LDA.fit_transform(dtm)
corpus['Topic']=topic_results.argmax(axis=1)
corpus['prob_topic']=topic_results[:,1]

In [176]:
corpus.to_pickle('../data/processed/amazon/corpus.gzip',compression='gzip')

In [177]:
corpus.head()

,cleaned_text,Target,Topic,prob_topic
0,book recommend friend finally read year late wow wait start addres...,1,1,0.996960
1,need question know child love impair identify mental illness behav...,1,1,0.996156
2,keep interest way definitely read jacob s mother teacher father da...,1,1,0.995366
3,c w gortner favorite author time wow queen release day novel local...,1,1,0.998420
4,love book like jk alex delaware series problem ending closure oh w...,1,1,0.995873


## IQR

In [178]:
q1, q3= np.percentile(corpus.loc[corpus.Topic==1,'prob_topic'],[25,75])

In [179]:
iqr = q3-q1
lower_bound = q1 -(1.5 * iqr) 
upper_bound = q3 +(1.5 * iqr) 

In [180]:
q1

0.9943309095570705

In [181]:
q3

0.9970910136602333

In [182]:
iqr

0.0027601041031628526

In [183]:
lower_bound

0.9901907534023262

In [184]:
upper_bound

1.0012311698149776

In [185]:
corpus.head()

,cleaned_text,Target,Topic,prob_topic
0,book recommend friend finally read year late wow wait start addres...,1,1,0.996960
1,need question know child love impair identify mental illness behav...,1,1,0.996156
2,keep interest way definitely read jacob s mother teacher father da...,1,1,0.995366
3,c w gortner favorite author time wow queen release day novel local...,1,1,0.998420
4,love book like jk alex delaware series problem ending closure oh w...,1,1,0.995873


In [186]:
corpus['outlier_iqr']=(corpus.Topic==1) & (corpus.prob_topic<lower_bound)

In [187]:
corpus.outlier_iqr.sum()

1837

In [ ]:
## Reorganización del corpus

In [188]:
corpus['new_target']=0

In [189]:
corpus.loc[corpus.outlier_iqr==True,'new_target']=2
corpus.loc[corpus.Topic==1 &(corpus.outlier_iqr==False),'new_target']=1
corpus.loc[(corpus.Topic==0) & (corpus.outlier_iqr==False),'new_target']=0

En este caso no han existido outliers...

In [192]:
joblib.dump(corpus,'../data/processed/amazon/corpus_reordered.final.pck')

['../data/processed/amazon/corpus_reordered.final.pck']

In [123]:
tsne.poof()

In [191]:
h2o.shutdown()

H2O session _sid_b2f0 closed.
